In [3]:
import random
import sqlite3

def generate_and_save_mTSP_instances_to_db(nr_instances, nr_cities, nr_salesmen = 1, coord_range=(0, 100), db_file="mTSP_instances.sqlite3"):

    # Connect to the database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Create the tables if they do not exist

    """Model metrics:
    average_distance: average distance between all cities
    stddev_distance: standard deviation of distances between all cities
    density: average distance between cities divided by the area of the bounding box
    salesmen_ratio: number of salesmen divided by the number of cities
    bounding_box_area: area of the bounding box containing all cities
    aspect_ratio: ratio of the width to the height of the bounding box
    spread: maximum distance between any two cities divided by the average distance
    cluster_compactness: average distance between cities divided by the number of cities
    mst_total_length: total length of the minimum spanning tree connecting all cities
    entropy_distance_matrix: entropy of the distance matrix between all cities
    """
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS instances (
            instance_id INTEGER PRIMARY KEY,
            nr_cities INTEGER,
            nr_salesmen INTEGER,
            average_distance REAL,  
            stddev_distance REAL,
            density REAL,
            salesmen_ratio REAL,       
            bounding_box_area REAL,
            aspect_ratio REAL,
            spread REAL,
            cluster_compactness REAL,
            mst_total_length REAL,
            entropy_distance_matrix REAL
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cities (
            city_id INTEGER,
            instance_id INTEGER,
            x INTEGER,
            y INTEGER,
            PRIMARY KEY (city_id, instance_id),
            FOREIGN KEY (instance_id) REFERENCES instances(instance_id)
        )
    """)

    # Find next instance_id
    cursor.execute("SELECT MAX(instance_id) FROM instances")
    result = cursor.fetchone()
    next_instance_id = result[0] + 1 if result[0] is not None else 1
  
    # Create and save instances
    for instance_id in range(next_instance_id, next_instance_id + nr_instances):

        # instace
        cursor.execute("INSERT INTO instances (instance_id, nr_cities, nr_salesmen) VALUES (?, ?, ?)",
                       (instance_id, nr_cities, nr_salesmen))

        # cities
        for city_id in range(nr_cities + 1):
            x = random.randint(coord_range[0], coord_range[1])
            y = random.randint(coord_range[0], coord_range[1])
            cursor.execute("INSERT INTO cities (city_id, instance_id, x, y) VALUES (?, ?, ?, ?)",
                           (city_id, instance_id, x, y))

    conn.commit()
    conn.close()

    print(f"{nr_instances} mTSP instances were created and added to {db_file}.")


generate_and_save_mTSP_instances_to_db(nr_instances=7, nr_cities=1000, nr_salesmen=10, coord_range=(0, 800), db_file="test_mTSP.sqlite3")

7 mTSP instances were created and added to test_mTSP.sqlite3.


In [4]:
# Calculate metrics for each instance and save them to the database
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.stats import entropy
import sqlite3
import math
import numpy as np

def compute_features_for_instance(city_coords, nr_cities, nr_salesmen):

    
    # Distanțe între toate perechile de orașe
    distances = pdist(city_coords)
    distance_matrix = squareform(distances)

    average_distance = sum(distances) / len(distances) if len(distances) > 0 else 0
    stddev_distance = math.sqrt(sum((d - average_distance) ** 2 for d in distances) / len(distances))


    # Bounding box
    min_x = min(city[0] for city in city_coords)
    min_y = min(city[1] for city in city_coords)
    max_x = max(city[0] for city in city_coords)
    max_y = max(city[1] for city in city_coords)
    width = max_x - min_x
    height = max_y - min_y
    bounding_box_area = width * height
    aspect_ratio = width / height if height != 0 else 0

    # Density
    density = average_distance / bounding_box_area if bounding_box_area != 0 else 0

    # Salesmen ratio
    salesmen_ratio = nr_salesmen / nr_cities if nr_cities != 0 else 0

    # Spread
    spread = max(distances) / average_distance if average_distance != 0 else 0

    # Cluster compactness
    cluster_compactness = average_distance / nr_cities if nr_cities != 0 else 0

    # MST
    mst = minimum_spanning_tree(distance_matrix)
    mst_total_length = mst.sum()

    # Entropy of the distance matrix
    hist, _ = np.histogram(distances, bins=20, density=True)
    entropy_distance_matrix = entropy(hist + 1e-9)  # evit log(0)

    return {
        'average_distance': average_distance,
        'stddev_distance': stddev_distance,
        'density': density,
        'salesmen_ratio': salesmen_ratio,
        'bounding_box_area': bounding_box_area,
        'aspect_ratio': aspect_ratio,
        'spread': spread,
        'cluster_compactness': cluster_compactness,
        'mst_total_length': float(mst_total_length),
        'entropy_distance_matrix': float(entropy_distance_matrix)
    }

# Use the database created above to compute metrics for each instance
db_file = "test_mTSP.sqlite3"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute("SELECT instance_id, nr_cities, nr_salesmen FROM instances")
instances = cursor.fetchall()

for instance_id, nr_cities, nr_salesmen in instances:
    cursor.execute("SELECT x, y FROM cities WHERE instance_id = ?", (instance_id,))
    city_coords = cursor.fetchall()
    metrics = compute_features_for_instance(city_coords, nr_cities, nr_salesmen)

    # Update the instance with the calculated metrics
    cursor.execute("""
        UPDATE instances
        SET average_distance = ?, stddev_distance = ?, density = ?, salesmen_ratio = ?, 
            bounding_box_area = ?, aspect_ratio = ?, spread = ?, cluster_compactness = ?, 
            mst_total_length = ?, entropy_distance_matrix = ?
        WHERE instance_id = ?
    """, (
        metrics['average_distance'], metrics['stddev_distance'], metrics['density'], metrics['salesmen_ratio'],
        metrics['bounding_box_area'], metrics['aspect_ratio'], metrics['spread'], metrics['cluster_compactness'],
        metrics['mst_total_length'], metrics['entropy_distance_matrix'], instance_id
    ))

conn.commit()
conn.close()

In [ ]:
import sqlite3

def delete_instance(instance_id, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:
        cursor.execute("DELETE FROM cities WHERE instance_id = ?", (instance_id,))
        cursor.execute("DELETE FROM algorithms WHERE instance_id = ?", (instance_id,))
        cursor.execute("DELETE FROM routes WHERE instance_id = ?", (instance_id,))

        cursor.execute("DELETE FROM instances WHERE instance_id = ?", (instance_id,))

        conn.commit()
        print(f"Instance {instance_id} and its associated data have been deleted.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
# delete_instance(instance_id=61, db_file="train_mTSP.sqlite3")
# for i in range(60,71):
#     delete_instance(instance_id=i, db_file="train_mTSP.sqlite3")


Instance 60 and its associated data have been deleted.
Instance 61 and its associated data have been deleted.
Instance 62 and its associated data have been deleted.
Instance 63 and its associated data have been deleted.
Instance 64 and its associated data have been deleted.
Instance 65 and its associated data have been deleted.
Instance 66 and its associated data have been deleted.
Instance 67 and its associated data have been deleted.
Instance 68 and its associated data have been deleted.
Instance 69 and its associated data have been deleted.
Instance 70 and its associated data have been deleted.
